In [17]:
import pickle
import numpy as np
import torch.utils.data as utils
import torch
from torchvision import transforms, models

In [2]:
def read_pickle_file(name):
    with open(name,'rb') as f:
        x = pickle.load(f)
    return x

In [3]:
train = read_pickle_file("data/Q2/train_CIFAR.pickle")
test = read_pickle_file("data/Q2/test_CIFAR.pickle")

In [4]:
X_train = train['X']
Y_train = train['Y']
X_test = test['X']
Y_test = test['Y']

In [5]:
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((10000, 3072), (10000,), (2000, 3072), (2000,))

In [6]:
X_train = X_train.reshape((-1,32,32,3))
X_test = X_test.reshape((-1,32,32,3))

In [7]:
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((10000, 32, 32, 3), (10000,), (2000, 32, 32, 3), (2000,))

In [8]:
Y_train_for_tensor = np.array([np.array([i]) for i in Y_train])
Y_test_for_tensor = np.array([np.array([i]) for i in Y_test])

In [9]:
scale =  transforms.Compose([transforms.ToPILImage(),transforms.Resize(224),transforms.ToTensor()])

In [10]:
scaled_X_train = [scale(i) for i in X_train]
scaled_X_test = [scale(i) for i in X_test]

In [11]:
tensorX_train = torch.stack([i for i in scaled_X_train])
tensorY_train = torch.stack([torch.Tensor(i) for i in Y_train_for_tensor])

In [12]:
tensorX_test = torch.stack([i for i in scaled_X_test])
tensorY_test = torch.stack([torch.Tensor(i) for i in Y_test_for_tensor])

In [13]:
train_dataset = utils.TensorDataset(tensorX_train,tensorY_train)
train_dataset_loader = utils.DataLoader(train_dataset)

In [14]:
test_dataset = utils.TensorDataset(tensorX_test,tensorY_test)
test_dataset_loader = utils.DataLoader(test_dataset)

In [16]:
len(train_dataset_loader),len(test_dataset_loader)

(10000, 2000)

In [18]:
net = models.AlexNet()
print(net)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [44]:
def get_feature_vector(dataloader):
    feature = []
    label = []
    count = 0
    for data in dataloader:
        if count%100 == 0:
            print(count)
        i,l = data
        feature.append(net.forward(i).tolist()[0]) 
        label.append(l.tolist()[0][0])
        count+=1
    return feature,label

In [45]:
train_feature,train_label = get_feature_vector(train_dataset_loader)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [51]:
tran_data = {'X':train_feature,'Y':train_label}

In [52]:
test_feature,test_label = get_feature_vector(test_dataset_loader)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [56]:
test_data = {'X':test_feature,'Y':test_label}

In [57]:
def dump_as_pickle(file_name,data):
    with open(file_name,'wb') as out_file:
        pickle.dump(data,out_file)

In [58]:
dump_as_pickle("data/Q2/train_feature_extracted.pickle",tran_data)

In [59]:
dump_as_pickle("data/Q2/test_feature_extracted.pickle",test_data)